In [3]:
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.restaurant_constants import *

from packages.restaurant.Restaurant import *
from packages.file_handler_package.file_handler import *

import os
import glob
import time
import pandas as pd
import ast
import json
import requests
import ast

In [4]:
path_to_res_merge_restaurant_phuket = os.path.join(fh.STORE_MERGE_RESTAURANT_SCRAPING, 'res_merge_restaurant', 'res_merge_restaurant_Phuket.csv')
merge_restaurant_phuket_df = pd.read_csv(path_to_res_merge_restaurant_phuket)

merge_restaurant_phuket_df = merge_restaurant_phuket_df.fillna('')
merge_restaurant_phuket_df

,name,description,priceRange,latitude,longitude,imgPath,phone,website,facility,type,address,province,district,subDistrict,province_code,district_code,sub_district_code,score,ratingCount
0,Tantitium Restaurant,,฿300.00 – ฿999.00,7.886630,98.385740,['https://dynamic-media-cdn.tripadvisor.com/me...,+66 81 797 3329,https://th.tripadvisor.com/Restaurant_Review-g...,"['Wifi ฟรี', 'การจอง', 'ซื้อกลับบ้าน', 'ดนตรีส...","['อาหารฟิวชั่น', 'อาหารไทย']","84, Dibuk Road, เมืองภูเก็ต, จังหวัดภูเก็ต 830...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,4.5,75
1,Day & Night of Phuket,,฿80.00 – ฿999.00,7.885968,98.391045,['https://dynamic-media-cdn.tripadvisor.com/me...,+66 76 219 788,https://th.tripadvisor.com/Restaurant_Review-g...,"['การจอง', 'การรับประทานอาหารแบบส่วนตัว', 'ที่...","['อาหารอิตาเลี่ยน', 'ร้านเหล้า/บาร์', 'สเต็ก',...","1 / 5 Dibuk Rd., ตลาดใหญ่, เมืองภูเก็ต, จังหวั...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,5.0,921
2,Palai Seafood,,,7.838537,98.364760,['https://dynamic-media-cdn.tripadvisor.com/me...,,https://th.tripadvisor.com/Restaurant_Review-g...,"['การจอง', 'ที่นั่ง', 'ที่นั่งกลางแจ้ง', 'บริก...","['อาหารทะเล', 'อาหารไทย']","Palai Beach, ฉลอง, เมืองภูเก็ต, จังหวัดภูเก็ต ไทย",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,3.5,55
3,Kuay Jub Yuan Moo Yo @ Samkong,,,7.905740,98.377300,['https://dynamic-media-cdn.tripadvisor.com/me...,+66 96 925 6463,https://th.tripadvisor.com/Restaurant_Review-g...,[],"['อาหารเวียดนาม', 'อาหารไทย']","27 Phuket Villas 1 Yaowarat 1 Road Taladyai, M...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,5.0,1
4,Huagu Chinese Cuisine,,,7.852317,98.331940,['https://dynamic-media-cdn.tripadvisor.com/me...,+66 81 968 1799,https://th.tripadvisor.com/Restaurant_Review-g...,[],['อาหารจีน'],"90/60 ถ. เจ้าฟ้าหมู่ 6 ต, ฉลอง, เมืองภูเก็ต, จ...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,4.5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Kucina Restaurant,,,7.940970,98.422890,[''],+66 76 352 148,https://th.tripadvisor.com/Restaurant_Review-g...,[],[],"51/7 หมู่ที่ 6 The Village Coconut Island, เกา...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,4.5,3
304,Home Steak,,,7.792972,98.321670,[''],+66 81 891 3264,https://th.tripadvisor.com/Restaurant_Review-g...,"['Wifi ฟรี', 'การจอง', 'ซื้อกลับบ้าน', 'ที่นั่...",['อาหารไทย'],"Saiyuan Rd. 50/10 Moo7, เมืองภูเก็ต, จังหวัดภู...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,4.5,55
305,ชาบูซัง,,,7.907419,98.373040,['https://dynamic-media-cdn.tripadvisor.com/me...,+66 61 954 2542,https://th.tripadvisor.com/Restaurant_Review-g...,['การจอง'],"['อาหารญี่ปุ่น', 'อาหารไทย']","141 หมู่ 5 ถนนเยาวราช ตำบลรัษฎา, เมืองภูเก็ต, ...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,5.0,9
306,Full Moon Brewworks - Taproom @ Phuket old town,,฿200.00 – ฿499.00,7.885442,98.390740,['https://dynamic-media-cdn.tripadvisor.com/me...,+66 92 326 9198,https://th.tripadvisor.com/Restaurant_Review-g...,"['Mastercard', 'Visa', 'Wifi ฟรี', 'ซื้อกลับบ้...","['ร้านเหล้า/บาร์', 'อาหารไทย']","32 Thep Krasattri Rd., Talad-yai, Muang Phuket...",ภูเก็ต,เมืองภูเก็ต,,83,8301,0,4.5,3


In [ ]:
def create_payload(row: pd.Series) -> dict:

    # openingHour_list_payload = [{ "day": key, "openingHour": val } for key, val in ast.literal_eval(row['openingHour']).items()]

    payload = {
        "name": row['name'],
        "type": ast.literal_eval(row['type']),
        "description": row['description'],
        "latitude": row['latitude'],
        "longitude": row['longitude'],
        "imgPath": ast.literal_eval(row['imgPath']),
        "phone": row['phone'],
        "website": row['website'],
        "priceRange": row['priceRange'],
        "facility": ast.literal_eval(row['facility']),
        "location": {
            "address": row['address'],
            "province": row['province'],
            "province_code": row['province_code'],
            "district": row['district'],
            "district_code": row['district_code'],
            "subDistrict": row['subDistrict'],
            "sub_district_code": row['sub_district_code']
        },
        "rating": {
            "score": row['score'],
            "ratingCount": row['ratingCount']
        }
    }

    return payload.copy()

In [ ]:
for Idx, cur_row in merge_restaurant_phuket_df.iterrows():

    cur_payload = create_payload(row=cur_row)

    try:
        API_ENDPOINT = f"{os.environ['TRIPWEAVER_API']}/api/restaurant/create"
        post_request = requests.post(url=API_ENDPOINT, json=cur_payload)
        print(post_request.text)
        
    except Exception as e:
        print("POST api failed at --> ", cur_payload['name'])
        print(e)

{"message":"An error occured while insert data restaurant SyntaxError: Unexpected token 'a', \"name=Tantit\"... is not valid JSON"}
{"message":"An error occured while insert data restaurant SyntaxError: Unexpected token 'a', \"name=Day+%2\"... is not valid JSON"}
{"message":"An error occured while insert data restaurant SyntaxError: Unexpected token 'a', \"name=Palai+\"... is not valid JSON"}
{"message":"An error occured while insert data restaurant SyntaxError: Unexpected token 'a', \"name=Kuay+J\"... is not valid JSON"}
{"message":"An error occured while insert data restaurant SyntaxError: Unexpected token 'a', \"name=Huagu+\"... is not valid JSON"}
{"message":"An error occured while insert data restaurant SyntaxError: Unexpected token 'a', \"name=House+\"... is not valid JSON"}
{"message":"An error occured while insert data restaurant SyntaxError: Unexpected token 'a', \"name=%E0%B8\"... is not valid JSON"}
{"message":"An error occured while insert data restaurant SyntaxError: Unexp